In [1]:
import os

# move to project root
while True:
    # get list of directories
    dirs = os.listdir()
    if "README.md" in dirs:
        break
    else:
        os.chdir("..")
print(os.getcwd())

/mnt/antares_raid/home/bramantyos/codes/multilang_timescale


In [2]:
import json
from typing import List, Dict

In [3]:
from src.settings import TrainerConfig, SubjectConfig, FeatureConfig, ResultConfig
from src.trainer import Trainer
from src.config import timescales

In [4]:
# logger = logging.getLogger('load_data')
# logger.setLevel('INFO')

In [5]:
# Params
lang_code = "en"
subject_id = "COL"
task = "reading"

use_bert = True

# Data

In [6]:
# BLING
## COL
feature_dir = ".temp/feature/joined"

col_en_data = ".temp/fmri/bling/COL/moth_reading_en_COL.hf5"
col_zh_data = ".temp/fmri/bling/COL/moth_reading_zh_COL.hf5"
fmri_data = col_en_data if not lang_code=="zh" else col_zh_data

In [7]:
lm_feature_path = ".temp/feature/timescales_mBERT_all_en.npz" if not lang_code=="zh" else ".temp/feature/timescales_mBERT_all_zh.npz"

if use_bert:
    lm_feature_path = ".temp/feature/timescales_BERT_all_en.npz"

sensory_feature_list = ['numwords', 'numletters', 'moten']

joined_feature_list = "-".join(sensory_feature_list)

sensory_feature_path = f"Baseline_bling_{task}_{lang_code}_{subject_id}_{joined_feature_list}.npz" 
sensory_feature_path = os.path.join(feature_dir, sensory_feature_path)

# Config

In [15]:

if lang_code == "zh":
    trim_start = 10
    trim_end = 12
else:
    trim_start=10
    trim_end = 10
    
sub_config = SubjectConfig(
    sub_id=subject_id,
    sub_fmri_train_test_path = fmri_data,
    lang_code=lang_code,
    sub_trim_start = trim_start,
    sub_trim_end = trim_end
)



In [16]:
trainer_config = TrainerConfig(
    backend="torch_cuda",
    result_save_dir = ".temp/result/bling",
    result_meta_save_dir = ".temp/result_meta/bling",
    n_targets_batch = 4096,
    n_targets_batch_refit = 2048,
    n_iter=100)

In [17]:
feature_config = FeatureConfig(
    lm_feature_type="mBERT",
    lm_feature_path=lm_feature_path,
    join_sensory_feature_path = sensory_feature_path,
)

In [18]:
config_dir = ".temp/config/bling"


In [19]:
sub_json = os.path.join(config_dir, "subject", f"{subject_id}_{lang_code}.json")
with open(sub_json, "w") as f:
    json.dump(sub_config.__dict__, f, indent=4)
    

In [20]:
feature_json = os.path.join(config_dir, "feature", f"mBERT_all_timescale_{lang_code}.json")

if use_bert:
    feature_json = os.path.join(config_dir, "feature", f"BERT_all_timescale_{lang_code}.json")  

with open(feature_json, "w") as f:
    json.dump(feature_config.__dict__, f, indent=4)

In [21]:
trainer_json = os.path.join(config_dir, "train", f"trainer_short.json")
with open(trainer_json, "w") as f:
    json.dump(trainer_config.__dict__, f, indent=4)

# Training

In [15]:
trainer = Trainer(sub_json, feature_json, trainer_json)

In [16]:
trainer.train()

Fitting model...
[........................................] 100% | 432.36 sec | 100 random sampling with cv | 
training took 443.3334701061249 seconds


In [17]:
trainer.refit_and_evaluate()

refitting...
predicting in batches...
computing scores...


/mnt/antares_raid/home/bramantyos/miniconda3/envs/vem/lib/python3.10/site-packages/himalaya/scoring.py:169: RuntimeWarning: divide by zero encountered in divide
  r2[fsi, :] = ((2 * inter - asst) / sst).sum(0)
/mnt/antares_raid/home/bramantyos/miniconda3/envs/vem/lib/python3.10/site-packages/himalaya/scoring.py:169: RuntimeWarning: invalid value encountered in divide
  r2[fsi, :] = ((2 * inter - asst) / sst).sum(0)
/mnt/antares_raid/home/bramantyos/miniconda3/envs/vem/lib/python3.10/site-packages/himalaya/scoring.py:61: RuntimeWarning: divide by zero encountered in divide
  r2 = 1. - error / var
/mnt/antares_raid/home/bramantyos/miniconda3/envs/vem/lib/python3.10/site-packages/himalaya/scoring.py:61: RuntimeWarning: invalid value encountered in divide
  r2 = 1. - error / var


computing permutation test...
computing timescale selectivity...
saving stat...


/mnt/antares_raid/home/bramantyos/codes/multilang_timescale/src/utils.py:151: RuntimeWarning: invalid value encountered in divide
  normalized_scores = np.nan_to_num(nz_scores / score_sum)


# Misc

In [ ]:
import numpy as np


In [ ]:
mbert = np.load(lm_feature_path, allow_pickle=True)

In [ ]:
mbert_train = mbert['train']

In [ ]:
mbert_train = mbert_train.tolist()

In [ ]:
mbert_train['2_4_words'].shape

In [ ]:
from src.config import train_stories, test_stories, train_stories_zh, test_stories_zh
from src.utils import load_dict, cook_responses

In [ ]:
data = load_dict(fmri_data)

In [ ]:
train_data, test_data = cook_responses(
    data,
    # test_runs=test_stories_zh,
    # train_runs=train_stories_zh,
    test_runs=test_stories,
    train_runs=train_stories,
    
    # trim_start_length=sub_config.sub_trim_start,
    # trim_end_length=sub_config.sub_trim_end,
    multiseries="average_across",
)


In [ ]:
train_data = np.nan_to_num(train_data)
test_data = np.nan_to_num(test_data[0])

In [ ]:
train_data.shape